In [1]:
import os
import sys
import time
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import torchvision
from torchvision import datasets
import torchvision.transforms as transforms

# Set up warnings
import warnings
warnings.filterwarnings(
    action='ignore',
    category=DeprecationWarning,
    module=r'.*'
)
warnings.filterwarnings(
    action='default',
    module=r'torch.ao.quantization'
)

# Specify random seed for repeatable results
torch.manual_seed(191009)

c:\Users\DELL\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\DELL\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
c:\Users\DELL\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
from torch.nn.utils import prune
from collections import OrderedDict
from models.googlenet import GoogleNet

model = GoogleNet()
model.load_state_dict(torch.load('googlenet-196-best.pth', map_location=torch.device('cpu')))
print(model)


GoogleNet(
  (prelayer): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (7): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
  )
  (a3): Inception(
    (b1): Sequential(
      (0): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (b2): Sequential(
      (0): Conv2d(192, 96, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(96, eps=1

In [3]:
model.state_dict().keys()

odict_keys(['prelayer.0.weight', 'prelayer.1.weight', 'prelayer.1.bias', 'prelayer.1.running_mean', 'prelayer.1.running_var', 'prelayer.1.num_batches_tracked', 'prelayer.3.weight', 'prelayer.4.weight', 'prelayer.4.bias', 'prelayer.4.running_mean', 'prelayer.4.running_var', 'prelayer.4.num_batches_tracked', 'prelayer.6.weight', 'prelayer.7.weight', 'prelayer.7.bias', 'prelayer.7.running_mean', 'prelayer.7.running_var', 'prelayer.7.num_batches_tracked', 'a3.b1.0.weight', 'a3.b1.0.bias', 'a3.b1.1.weight', 'a3.b1.1.bias', 'a3.b1.1.running_mean', 'a3.b1.1.running_var', 'a3.b1.1.num_batches_tracked', 'a3.b2.0.weight', 'a3.b2.0.bias', 'a3.b2.1.weight', 'a3.b2.1.bias', 'a3.b2.1.running_mean', 'a3.b2.1.running_var', 'a3.b2.1.num_batches_tracked', 'a3.b2.3.weight', 'a3.b2.3.bias', 'a3.b2.4.weight', 'a3.b2.4.bias', 'a3.b2.4.running_mean', 'a3.b2.4.running_var', 'a3.b2.4.num_batches_tracked', 'a3.b3.0.weight', 'a3.b3.0.bias', 'a3.b3.1.weight', 'a3.b3.1.bias', 'a3.b3.1.running_mean', 'a3.b3.1.runni

In [4]:
model.named_modules

<bound method Module.named_modules of GoogleNet(
  (prelayer): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (7): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
  )
  (a3): Inception(
    (b1): Sequential(
      (0): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (b2): Sequential(
      (0): Conv2d(192, 96, kernel_size=(1, 1), stride=(1

In [5]:
'''for name, module in model.named_modules():
    for mini_name, mini_module in module.named_modules():
        # prune 20% of connections in all 2D-conv layers
        if isinstance(mini_module, torch.nn.Conv2d):
            print(f"Sparsity in {mini_name} before: {100. * float(torch.sum(mini_module.weight == 0))/ float(mini_module.weight.nelement()):.2f}%")
            prune.l1_unstructured(mini_module, name='weight', amount=0.4)
            prune.remove(mini_module, 'weight')
            print(f"Sparsity in {mini_name} after: {100. * float(torch.sum(mini_module.weight == 0))/ float(mini_module.weight.nelement()):.2f}%")
        # prune 40% of connections in all linear layers

print(dict(model.named_buffers()).keys())  # to verify that all masks exist'''

'for name, module in model.named_modules():\n    for mini_name, mini_module in module.named_modules():\n        # prune 20% of connections in all 2D-conv layers\n        if isinstance(mini_module, torch.nn.Conv2d):\n            print(f"Sparsity in {mini_name} before: {100. * float(torch.sum(mini_module.weight == 0))/ float(mini_module.weight.nelement()):.2f}%")\n            prune.l1_unstructured(mini_module, name=\'weight\', amount=0.4)\n            prune.remove(mini_module, \'weight\')\n            print(f"Sparsity in {mini_name} after: {100. * float(torch.sum(mini_module.weight == 0))/ float(mini_module.weight.nelement()):.2f}%")\n        # prune 40% of connections in all linear layers\n\nprint(dict(model.named_buffers()).keys())  # to verify that all masks exist'

In [6]:
modules_to_fuse = [
    ['prelayer.0', 'prelayer.1', 'prelayer.2'],
    ['prelayer.3', 'prelayer.4', 'prelayer.5'],
    ['prelayer.6', 'prelayer.7', 'prelayer.8'],

    ['a3.b1.0', 'a3.b1.1', 'a3.b1.2'],
    ['a3.b2.0', 'a3.b2.1', 'a3.b2.2'],
    ['a3.b2.3', 'a3.b2.4', 'a3.b2.5'],
    ['a3.b3.0', 'a3.b3.1', 'a3.b3.2'],
    ['a3.b3.3', 'a3.b3.4', 'a3.b3.5'],
    ['a3.b3.6', 'a3.b3.7', 'a3.b3.8'],
    ['a3.b4.1', 'a3.b4.2', 'a3.b4.3'],

    ['b3.b1.0', 'b3.b1.1', 'b3.b1.2'],
    ['b3.b2.0', 'b3.b2.1', 'b3.b2.2'],
    ['b3.b2.3', 'b3.b2.4', 'b3.b2.5'],
    ['b3.b3.0', 'b3.b3.1', 'b3.b3.2'],
    ['b3.b3.3', 'b3.b3.4', 'b3.b3.5'],
    ['b3.b3.6', 'b3.b3.7', 'b3.b3.8'],
    ['b3.b4.1', 'b3.b4.2', 'b3.b4.3'],

    ['a4.b1.0', 'a4.b1.1', 'a4.b1.2'],
    ['a4.b2.0', 'a4.b2.1', 'a4.b2.2'],
    ['a4.b2.3', 'a4.b2.4', 'a4.b2.5'],
    ['a4.b3.0', 'a4.b3.1', 'a4.b3.2'],
    ['a4.b3.3', 'a4.b3.4', 'a4.b3.5'],
    ['a4.b3.6', 'a4.b3.7', 'a4.b3.8'],
    ['a4.b4.1', 'a4.b4.2', 'a4.b4.3'],

    ['b4.b1.0', 'b4.b1.1', 'b4.b1.2'],
    ['b4.b2.0', 'b4.b2.1', 'b4.b2.2'],
    ['b4.b2.3', 'b4.b2.4', 'b4.b2.5'],
    ['b4.b3.0', 'b4.b3.1', 'b4.b3.2'],
    ['b4.b3.3', 'b4.b3.4', 'b4.b3.5'],
    ['b4.b3.6', 'b4.b3.7', 'b4.b3.8'],
    ['b4.b4.1', 'b4.b4.2', 'b4.b4.3'],

    ['c4.b1.0', 'c4.b1.1', 'c4.b1.2'],
    ['c4.b2.0', 'c4.b2.1', 'c4.b2.2'],
    ['c4.b2.3', 'c4.b2.4', 'c4.b2.5'],
    ['c4.b3.0', 'c4.b3.1', 'c4.b3.2'],
    ['c4.b3.3', 'c4.b3.4', 'c4.b3.5'],
    ['c4.b3.6', 'c4.b3.7', 'c4.b3.8'],
    ['c4.b4.1', 'c4.b4.2', 'c4.b4.3'],

    ['d4.b1.0', 'd4.b1.1', 'd4.b1.2'],
    ['d4.b2.0', 'd4.b2.1', 'd4.b2.2'],
    ['d4.b2.3', 'd4.b2.4', 'd4.b2.5'],
    ['d4.b3.0', 'd4.b3.1', 'd4.b3.2'],
    ['d4.b3.3', 'd4.b3.4', 'd4.b3.5'],
    ['d4.b3.6', 'd4.b3.7', 'd4.b3.8'],
    ['d4.b4.1', 'd4.b4.2', 'd4.b4.3'],

    ['e4.b1.0', 'e4.b1.1', 'e4.b1.2'],
    ['e4.b2.0', 'e4.b2.1', 'e4.b2.2'],
    ['e4.b2.3', 'e4.b2.4', 'e4.b2.5'],
    ['e4.b3.0', 'e4.b3.1', 'e4.b3.2'],
    ['e4.b3.3', 'e4.b3.4', 'e4.b3.5'],
    ['e4.b3.6', 'e4.b3.7', 'e4.b3.8'],
    ['e4.b4.1', 'e4.b4.2', 'e4.b4.3'],

    ['a5.b1.0', 'a5.b1.1', 'a5.b1.2'],
    ['a5.b2.0', 'a5.b2.1', 'a5.b2.2'],
    ['a5.b2.3', 'a5.b2.4', 'a5.b2.5'],
    ['a5.b3.0', 'a5.b3.1', 'a5.b3.2'],
    ['a5.b3.3', 'a5.b3.4', 'a5.b3.5'],
    ['a5.b3.6', 'a5.b3.7', 'a5.b3.8'],
    ['a5.b4.1', 'a5.b4.2', 'a5.b4.3'],

    ['b5.b1.0', 'b5.b1.1', 'b5.b1.2'],
    ['b5.b2.0', 'b5.b2.1', 'b5.b2.2'],
    ['b5.b2.3', 'b5.b2.4', 'b5.b2.5'],
    ['b5.b3.0', 'b5.b3.1', 'b5.b3.2'],
    ['b5.b3.3', 'b5.b3.4', 'b5.b3.5'],
    ['b5.b3.6', 'b5.b3.7', 'b5.b3.8'],
    ['b5.b4.1', 'b5.b4.2', 'b5.b4.3']
    ]

## Quantization using Static Quantization

In [7]:
from models.quantized_googlenet import Quantized_Googlenet

quantized_model = Quantized_Googlenet()
quantized_model.load_state_dict(torch.load('googlenet-196-best.pth', map_location=torch.device('cpu')))


quantized_model.eval()

quantized_model.qconfig = torch.ao.quantization.get_default_qconfig('x86')

quantized_model_fused = torch.ao.quantization.fuse_modules(quantized_model, modules_to_fuse)

quantized_model = torch.ao.quantization.prepare(quantized_model_fused)

model_int8 = torch.ao.quantization.convert(quantized_model)


c:\Users\DELL\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\ao\quantization\observer.py:214: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(
c:\Users\DELL\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\ao\quantization\observer.py:1209: UserWarning: must run observer before calling calculate_qparams.                                    Returning default scale and zero point 
  warnings.warn(


In [8]:
model_int8

Quantized_Googlenet(
  (prelayer): Sequential(
    (0): QuantizedConvReLU2d(3, 64, kernel_size=(3, 3), stride=(1, 1), scale=1.0, zero_point=0, padding=(1, 1))
    (1): Identity()
    (2): Identity()
    (3): QuantizedConvReLU2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=1.0, zero_point=0, padding=(1, 1))
    (4): Identity()
    (5): Identity()
    (6): QuantizedConvReLU2d(64, 192, kernel_size=(3, 3), stride=(1, 1), scale=1.0, zero_point=0, padding=(1, 1))
    (7): Identity()
    (8): Identity()
  )
  (a3): Inception(
    (b1): Sequential(
      (0): QuantizedConvReLU2d(192, 64, kernel_size=(1, 1), stride=(1, 1), scale=1.0, zero_point=0)
      (1): Identity()
      (2): Identity()
    )
    (b2): Sequential(
      (0): QuantizedConvReLU2d(192, 96, kernel_size=(1, 1), stride=(1, 1), scale=1.0, zero_point=0)
      (1): Identity()
      (2): Identity()
      (3): QuantizedConvReLU2d(96, 128, kernel_size=(3, 3), stride=(1, 1), scale=1.0, zero_point=0, padding=(1, 1))
      (4): Identi

In [9]:
torch.save(model_int8.state_dict(), 'quantized_googlenet.pth')